In [14]:
import numpy as np
import pandas as pd

In [15]:
mturk = pd.read_csv("../../data/labels/mturk_rusk_11-14.csv")

mturk = mturk[['WorkerId', 
               'Input.image_url', 
               'Answer.Extent', 
               'Answer.option1.on', 
               'Answer.option2.on', 
               'Answer.option3.on']]

In [16]:
mturk.head()

,WorkerId,Input.image_url,Answer.Extent,Answer.option1.on,Answer.option2.on,Answer.option3.on
0,A1BZMONU8QDPFQ,bigiron_1053.jpg,2,True,False,False
1,A1Q8PPQQCWGY0D,bigiron_1053.jpg,3,True,False,False
2,A2A1E85FL1JAB9,bigiron_1053.jpg,1,True,False,False
3,A3O0UKK3YYWWCF,bigiron_DA7311.jpg,2,True,False,False
4,A2OP2ZANIM5ZV,bigiron_DA7311.jpg,2,True,False,False


In [17]:
imageID_all = mturk["Input.image_url"].unique()

In [18]:
### image ID with 3 differnt answers

imageID_diff = []
for iid in imageID_all:
    temp = mturk[mturk["Input.image_url"] == iid]
    a = temp.iloc[0]
    b = temp.iloc[1]
    c = temp.iloc[2]
    if not ((a == b)[2:].all() or (a == c)[2:].all() or (b == c)[2:].all()):
        imageID_diff.append(iid)
len(imageID_diff)

114

In [19]:
### (WorkerId, ImageId) pairs with illegal answers (option 1 in Q1 but not 0 in Q2)

illegal_pairs = []
for i, row in mturk.iterrows():
    if (row['Answer.option1.on'] and row['Answer.Extent'] == 0) or (not row['Answer.option1.on'] and row['Answer.Extent'] != 0):
        illegal_pairs.append((row["WorkerId"], row["Input.image_url"]))
len(illegal_pairs)

47

In [20]:
mturk_clean = mturk.copy()
drops = []
for i, row in mturk_clean.iterrows():
    pairs = (row["WorkerId"], row["Input.image_url"])
    imageId = row["Input.image_url"]
    if pairs in illegal_pairs or imageId in imageID_diff:
        drops.append(i)
mturk_clean.drop(labels=drops, axis=0, inplace=True)
len(mturk_clean)

1135

In [21]:
from collections import Counter
Counter(mturk_clean.groupby("Input.image_url").agg(lambda x: len(x))["WorkerId"])

### 365 images have all 3 "legal" workers and "majority" results

Counter({3: 365, 2: 20})

In [22]:
### distribution of "true" labels

## if we only use images wiht 3 workers
three_workers = (mturk_clean.groupby("Input.image_url").agg(lambda x: len(x))["WorkerId"] == 3)
three_workers_imageId = three_workers.index[three_workers.values].tolist()
mturk_clean_3 = mturk_clean[mturk_clean["Input.image_url"].isin(three_workers_imageId)]
assert len(mturk_clean_3) == 365 * 3
print(Counter(mturk_clean_3["Answer.Extent"]))

## if we only use all cleaned labels
print(Counter(mturk_clean["Answer.Extent"]))

Counter({0: 427, 1: 257, 2: 234, 3: 177})
Counter({0: 455, 1: 263, 2: 236, 3: 181})


In [83]:
### Extract true labels

true_labels = mturk_clean_3.groupby("Input.image_url")["Answer.Extent"].agg(pd.Series.mode)
true_labels.rename("three", inplace=True)
true_labels.rename_axis("img", inplace=True)
true_labels = pd.DataFrame(data=true_labels)
true_labels = true_labels[true_labels["three"] != 0]
true_labels = true_labels[true_labels.index != "file_list.csv"]
true_labels.to_csv("../../data/labels/mturk_rusk_11-14_truth.csv")

In [84]:
len(true_labels[true_labels["three"] != 0])

218